In [1]:
num_encoder_tokens = 38

In [2]:
num_decoder_tokens = 38

In [3]:
target_token_index = {'\t': 0,
 '\n': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 'A': 12,
 'B': 13,
 'C': 14,
 'D': 15,
 'a': 16,
 'c': 17,
 'd': 18,
 'e': 19,
 'f': 20,
 'g': 21,
 'h': 22,
 'j': 23,
 'k': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'q': 30,
 'r': 31,
 's': 32,
 't': 33,
 'u': 34,
 'v': 35,
 'y': 36,
 'z': 37}

In [4]:
input_token_index = {'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'A': 10,
 'B': 11,
 'C': 12,
 'D': 13,
 'a': 14,
 'b': 15,
 'c': 16,
 'd': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'k': 23,
 'l': 24,
 'm': 25,
 'n': 26,
 'o': 27,
 'p': 28,
 'q': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'y': 36,
 'z': 37}

In [5]:
reverse_target_char_index = {0: '\t',
 1: '\n',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9',
 12: 'A',
 13: 'B',
 14: 'C',
 15: 'D',
 16: 'a',
 17: 'c',
 18: 'd',
 19: 'e',
 20: 'f',
 21: 'g',
 22: 'h',
 23: 'j',
 24: 'k',
 25: 'l',
 26: 'm',
 27: 'n',
 28: 'o',
 29: 'p',
 30: 'q',
 31: 'r',
 32: 's',
 33: 't',
 34: 'u',
 35: 'v',
 36: 'y',
 37: 'z'}

In [6]:
max_decoder_seq_length = 28

In [7]:
from keras.models import load_model

In [10]:
encoder_model = load_model('diff_enc.tf')

In [11]:
decoder_model = load_model('diff_dec.tf')

In [12]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    # Loop untill we recieve a stop sign
    while not stop_condition:
        # Get output and internal states of the decoder 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Get the predicted token (the token with the highest score)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Get the character belonging to the token
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Append char to output
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [13]:
def lettoword(let):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', 'E': 'a', 'I': 'b', '(+)': 'c', '(-)': 'd', "Y'": 'e', 'abs': 'f', 'acos': 'g', 'acosh': 'h', 'acoth': 'i', '+': 'j', 'asin': 'k', 'asinh': 'l', 'atan': 'm', 'atanh': 'n', 'cos': 'o', 'cosh': 'p', '/': 'q', 'exp': 'r', 'ln': 's', '*': 't', 'pi': 'u', 'pow': 'v', 'sign': 'w', 'sin': 'x', 'sinh': 'y', 'sqrt': 'z', '-': 'A', 'tan': 'B', 'tanh': 'C', 'x': 'D'}
    rev = {v:k for k, v in my_dict.items()}
    let = list(let)
    ret = []
    for key in let:
        ret.append(rev[key])
    return " ".join(ret)
        
def wordtolet(word):
    my_dict = {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', 'E': 'a', 'I': 'b', '(+)': 'c', '(-)': 'd', "Y'": 'e', 'abs': 'f', 'acos': 'g', 'acosh': 'h', 'acoth': 'i', '+': 'j', 'asin': 'k', 'asinh': 'l', 'atan': 'm', 'atanh': 'n', 'cos': 'o', 'cosh': 'p', '/': 'q', 'exp': 'r', 'ln': 's', '*': 't', 'pi': 'u', 'pow': 'v', 'sign': 'w', 'sin': 'x', 'sinh': 'y', 'sqrt': 'z', '-': 'A', 'tan': 'B', 'tanh': 'C', 'x': 'D'}
    word = word.split()
    ret = []
    for key in word:
        ret.append(my_dict[key])
    return "".join(ret)

In [14]:
def solve(text):
    my_text = wordtolet(text)
    placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))
    for i, char in enumerate(my_text):
        #print(i,char, input_token_index[char])
        placeholder[0,i,input_token_index[char]] = 1
    sol = decode_sequence(placeholder)[:-1]
    sol = lettoword(sol)
    return sol

In [19]:
def print_examples():
    train_lines = open('minialpha2.train').read().split('\n')
    for train_line in train_lines[5:6]:
        train_target_eq, train_input_eq = train_line.split('\t')
        train_input_eq = lettoword(train_input_eq)
        train_target_eq = lettoword(train_target_eq)
        #if(solve(train_input_eq) == train_target_eq):
        print("\nInput: " + train_input_eq + "\nPrediction: " + solve(train_input_eq) + "\nTarget: " + train_target_eq)

In [17]:
import numpy as np

In [18]:
print_examples()


Input: + pow x (+) 2 * / (+) 1 (+) 3 pow x (+) 3
Prediction: - Y' * x + (+) 1 + x * (+) 2 pow x (+) 2
Target: - Y' + pow x (+) 2 * (+) 2 x


In [21]:
print_examples()


Input: * / (+) 4 (+) 3 pow x (+) 3
Prediction: - Y' * x + (+) 1 + x * (+) 2 pow x (+) 2
Target: - Y' * (+) 4 pow x (+) 2
